## Load essentials

In [2]:
%run /Articles/helpers/Save_Model

In [3]:
%run /Articles/helpers/Baseline

In [4]:
%run /Articles/helpers/Test_Model

## Create Baseline

In [6]:
ami_df = table_to_df("merptrain_article")
ami_df = prep_df(ami_df)

In [7]:
vec, rf_model, predictions, report =  create_baseline(ami_df, "Tokenized", 3)

In [8]:
df = table_to_df("article_dataset_csv")

In [9]:
display(df)

In [10]:
dftest, df_prediction2, rouge_array = test_model_rf(
  rfmodel = rf_model, 
  dftest = False, 
  reload = True, 
  summary_table = "article_dataset_csv", 
  start = 2125, 
  stop = 2225, 
  threshold = 0.5,
  vec = vec
)

In [11]:
save(log = log, runname = "automated RF generation", modelname = "RF Baseline model", model = rf_model, rouge_array = rouge_array, baseline = True)

In [12]:
display(df_prediction2)

In [13]:
print(df_prediction2.iloc[8,2])

## Create Neural Networks

In [15]:
complete_df, df_sentences_pos, embedding_index = load_data(complete_table="article_sentences_df", train_table="meta_article_sentences_df_train2")

In [16]:
embedding_index

In [17]:
max_sequence = 1000
max_words = 30000
embedding_dim = 100

In [18]:
%run /Articles/helpers/Training_Model_2

In [19]:
model, history, cp, log = make_model(
                                modeltype = "CNN",
                                complete_df = complete_df, 
                                df_sentences_pos = df_sentences_pos, 
                                textcolumn_complete = 2, 
                                textcolumn_train = 2, 
                                labelcolumn_train = 3, 
                                meta_train_start = 4,
                                meta_train_stop = 6,
                                max_sequence = max_sequence,
                                max_words = max_words,
                                embedding_dim = embedding_dim,
                                val_split = 0.1,
                                embedding_index = embedding_index,
                                classweight_neg = 1.,
                                classweight_pos = 1.,
                                epoch = 3, 
                                glove_emblayer = False)


In [20]:
model = mlflow.keras.load_model("runs:/9a7c607830434cd0892d727a4d9e70d7" + "/MERP Article CNN model")

In [21]:
dftest, df_prediction, rouge_array = test_model_meta(model = model, dftest = dftest, reload = False, summary_table = "article_dataset_csv", sentence_table = "article_sentences_df", start = 2125, stop = 2225, max_sequence = max_sequence, max_words = max_words, embedding_dim = embedding_dim, threshold = 0.59)

In [22]:
save(log = log, runname = "Article automated LSTM generation", modelname = "Article LSTM model", model = model, rouge_array = rouge_array)

In [23]:
display(df_prediction)